In [8]:
import numpy as np
import pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import random

In [15]:
seed = 47
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

# 合并train_level study
train_level_df = pd.read_csv('/datadisk/kg/siim/tmp/train_image_level.csv')
train_study_df = pd.read_csv('/datadisk/kg/siim/tmp/train_study_level.csv')

train_study_df['id'] = train_study_df['id'].apply(lambda x: x[:-6])
train_study_df.rename(columns={'id': 'StudyInstanceUID'}, inplace=True)

train_level_df = train_level_df.merge(train_study_df, on='StudyInstanceUID')

# 合并meta
meta_df = pd.read_csv('/datadisk/kg/siim/tmp/meta.csv')
meta_df = meta_df[meta_df.split == 'train']
meta_df['image_id'] = meta_df['image_id'].apply(lambda x: x + '_image')
meta_df = meta_df.rename(columns={'image_id': 'id'})
meta_df.drop(columns=['split'], inplace=True)
train_level_df = train_level_df.merge(meta_df, on='id')

#  添加path
train_path = '/datadisk/kg/siim/tmp/train/'
train_level_df['path'] = train_level_df['id'].apply(lambda x: train_path + x[:-6] + '.png')

# 添加fold
gkf = GroupKFold(n_splits=4)
gkf = gkf.split(train_level_df, groups=train_level_df.StudyInstanceUID.tolist())
train_level_df['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(gkf):
    train_level_df.loc[val_idx, 'fold'] = fold

# 保存完整的train_level_df
train_level_df.to_csv('tmp/train_df_ano.csv', index=False)

In [16]:
train_df = pd.read_csv('/datadisk/kg/siim/tmp/train_df.csv')
train_ano_df = pd.read_csv('/datadisk/kg/siim/tmp/train_df_ano.csv')
train_df['fold']

0       4
1       0
2       4
3       1
4       3
       ..
6329    3
6330    3
6331    2
6332    1
6333    3
Name: fold, Length: 6334, dtype: int64

In [17]:
train_ano_df['fold']

0       2
1       1
2       2
3       2
4       2
       ..
6329    0
6330    3
6331    1
6332    3
6333    0
Name: fold, Length: 6334, dtype: int64